In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import optuna
from hyperopt import fmin, tpe, hp
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import category_encoders as ce
import optuna
from lightgbm import LGBMClassifier
from optuna.samplers import TPESampler

In [ ]:
file_path_train = r'C:\Users\mirko\Desktop\train.csv'
file_path_test = r'C:\Users\mirko\Desktop\test.csv'
train = pd.read_csv(file_path_train)
test = pd.read_csv(file_path_test)

In [ ]:
description = pd.DataFrame(index = list(train))
description['type'] = train.dtypes
description['count'] = train.count()
description['nunique'] = train.nunique()
description['null'] = train.isnull().sum()
description['min'] = train.min()
description['max'] = train.max()
description

In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# to_be_min_max_scaled = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

# min_max_scaler = MinMaxScaler()
# train[to_be_min_max_scaled] = min_max_scaler.fit_transform(train[to_be_min_max_scaled])
# test[to_be_min_max_scaled] = min_max_scaler.transform(test[to_be_min_max_scaled])


In [ ]:
categorical_columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'MTRANS', 'CALC']

category_mapping = {'Frequently' : 'Frequently', 'Sometimes' : 'Sometimes', 'no' : 'no', 'Always' : 'Frequently'}

train['CALC'] = train['CALC'].map(category_mapping).astype('category')
test['CALC'] = test['CALC'].map(category_mapping).astype('category')

In [ ]:
# train = pd.get_dummies(train, columns=categorical_columns)

# test = pd.get_dummies(test, columns=categorical_columns)
# train.head()

In [ ]:
train.dtypes

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()

# train['NObeyesdad'] = label_encoder.fit_transform(train['NObeyesdad'])

In [ ]:
X = train.drop(['id', 'NObeyesdad'], axis=1)
y = train.pop('NObeyesdad')

In [ ]:
classes = np.unique(y)

weights = compute_class_weight('balanced', classes=classes, y=y)

weights

class_weights = {
    0.0 : 1.17535813,
    1.0 : 0.96217669,
    2.0 : 1.01904762,
    3.0 : 0.91300141,
    4.0 : 0.73292847,
    5.0 : 1.22184943,
    6.0 : 1.17582418
}

In [ ]:
random_states_to_try = list(range(1, 1001))

best_accuracy = 0.0
best_random_state = 0.0



for random_state in random_states_to_try:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    for col in categorical_columns:
        X_train[col] = X_train[col].astype('category')
        X_val[col] = X_val[col].astype('category')
    
    lgb_params = {'objective' : 'multiclassova',
                 'learning_rate': 0.017496510857138103,
                  'n_estimators': 846,
                  'reg_alpha': 0.9339063703383828,
                  'reg_lambda': 0.23045254190652778,
                  'max_depth': 10,
                  'colsample_bytree': 0.6571224806269116,
                  'subsample': 0.6204128913658613,
                  'num_leaves': 33,
                  'boosting_type': 'gbdt',
                  'num_class': 7,
                  'random_state': 42,
                  'metric': 'multi_logloss',
                  'class_weight': 'balanced'
                 }
    
    lgb_model_final = LGBMClassifier(**lgb_params)
    lgb_model_final.fit(X_train, y_train)
    
    accuracy = accuracy_score(y_val, lgb_model_final.predict(X_val))
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_random_state = random_state
    
    print(f'Random State: {random_state}, Accuracy: {accuracy}')
    
print(f"Best Accuracy: {best_accuracy} achieved with random state: {best_random_state}")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=283)

for col in categorical_columns:
        X_train[col] = X_train[col].astype('category')
        X_val[col] = X_val[col].astype('category')

# LightGBM 

In [ ]:
# def optimization_function_lgb(trial):
    
    
    
    
    
#     lgb_params= {
#         'boosting_type' : 'gbdt',
#         'num_class' : 7,
#         'random_state' : 42,
#         'metric' : 'multi_logloss',
#         'class_weight' : 'balanced',
#         'objective' : trial.suggest_categorical('objective', ['multiclass', 'multiclassova']),
#         'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.05),
#         'n_estimators' : trial.suggest_int('n_estimators', 400, 1000),
#         'reg_alpha' : trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
#         'reg_lambda' : trial.suggest_loguniform('reg_lambda', 1e-1, 10.0),
#         'max_depth' : trial.suggest_int('max_depth', 5, 15),
#         'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.3, 0.9),
#         'subsample' : trial.suggest_float('subsample', 0.5, 1.0),
#         'num_leaves' : trial.suggest_int('num_leaves', 10, 50)
#     }
    
#     lgb_model = LGBMClassifier(**lgb_params)
    
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
#     accuracy = cross_val_score(lgb_model, X_train, y_train, cv=skf, scoring='accuracy')
#     print('='*50, '\n Validation Accuracy: ', accuracy.mean())
    
#     lgb_model.fit(X_train, y_train)
    
#     acc = accuracy_score(y_val, lgb_model.predict(X_val))
    
#     return acc

In [ ]:
# sampler = optuna.samplers.TPESampler(seed=42)

# study = optuna.create_study(direction='maximize', sampler=sampler)

# study.optimize(optimization_function_lgb, n_trials=50)

# best_params = study.best_params

# print('='*50)
# print(best_params)

# ran_optuna=True

In [ ]:
# lgb_params = study.best_trial.params

# fixed_params_lgb = {
#     'boosting_type' : 'gbdt',
#     'num_class' : 7,
#     'random_state' : 42,
#     'metric' : 'multi_logloss',
#     'class_weight' : 'balanced'
# }

# for i in fixed_params_lgb.keys():
#     lgb_params[i] = fixed_params_lgb[i]
    
# lgb_params

In [ ]:
# lgb_params = {'objective': 'multiclass',
#  'learning_rate': 0.021685785941408728,
#  'n_estimators': 620,
#  'reg_alpha': 0.06672367170464207,
#  'reg_lambda': 3.7183641805732095,
#  'max_depth': 7,
#  'colsample_bytree': 0.608540663048167,
#  'subsample': 0.7962072844310213,
#  'num_leaves': 11,
#  'boosting_type': 'gbdt',
#  'num_class': 7,
#  'random_state': 42,
#  'metric': 'multi_logloss',
#  'class_weight': 'balanced'}

# lgb_model_final = LGBMClassifier(**lgb_params)

# lgb_model_final.fit(X_train, y_train)

# accuracy = accuracy_score(y_val, lgb_model_final.predict(X_val))
# print(f'Accuracy: ', accuracy)

In [ ]:
lgb_params = {'objective': 'multiclassova',
 'learning_rate': 0.017496510857138103,
 'n_estimators': 846,
 'reg_alpha': 0.9339063703383828,
 'reg_lambda': 0.23045254190652778,
 'max_depth': 10,
 'colsample_bytree': 0.6571224806269116,
 'subsample': 0.6204128913658613,
 'num_leaves': 33,
 'boosting_type': 'gbdt',
 'num_class': 7,
 'random_state': 42,
 'metric': 'multi_logloss'}

lgb_model_final = LGBMClassifier(**lgb_params)

lgb_model_final.fit(X_train, y_train)

accuracy = accuracy_score(y_val, lgb_model_final.predict(X_val))
print(f'Accuracy: ', accuracy)

In [ ]:
test_noid = test.drop('id', axis=1)

for col in categorical_columns:
    test_noid[col] = test_noid[col].astype('category')

predictions = lgb_model_final.predict(test_noid)
predictions_labels = label_encoder.inverse_transform(predictions)

In [ ]:
submission = pd.DataFrame({
    'id' : test.id,
    'NObeyesdad' : predictions_labels
})

submission.to_csv('submission.csv', index=False)

submission.head()

In [ ]:
download_link = f'<a href="submission.csv" download> Click here to download the submission file</a>'
from IPython.display import HTML
HTML(download_link)